# Demo all Thunderbird WPS Processes

This notebook demonstrates how the scripts from PCIC's [climate-explorer-data-prep](https://github.com/pacificclimate/climate-explorer-data-prep/blob/master/README.md) repository are used as Web Processing Service (WPS) processes in thunderbird. Before a process can be run, an instance of `thunderbird` must be activated by executing `thunderbird start` on a terminal. Each process takes netcdf files as input and, unless otherwise stated, outputs a metalink between the resulting netcdf files to allow them to be downloaded. Netcdf files from local storage or an opendap server can be used as input.

In [1]:
from birdy import WPSClient
import requests
from pkg_resources import resource_filename
from bs4 import BeautifulSoup

In [2]:
import os 
if os.path.basename(os.getcwd()) != "thunderbird": # Ensure current directory is always 'thunderbird' for 'resource_filename' to work
    os.chdir('../')

In [3]:
# Set up wps application
url = 'http://localhost:5001/wps'
thunderbird = WPSClient(url=url)

In [4]:
# Check wps info
thunderbird?

Type:            WPSClient
String form:     <birdy.client.base.WPSClient object at 0x7f7288307978>
File:            ~/github/pcic/thunderbird/notebooks/demo_venv/lib/python3.6/site-packages/birdy/client/base.py
Docstring:      
A Web Processing Service for Climate Explorer data preparation

Processes
---------

generate_climos
    Generate files containing climatological means from input files of daily, monthly, or yearly data that adhere to the PCIC metadata standard (and consequently to CMIP5 and CF standards).

generate_prsn
    Generate precipitation as snow file from precipitation and minimum/maximum temperature data

update_metadata
    Update file containing missing, invalid, or incorrectly named global or variable metadata attributes

split_merged_climos
    Split climo means files into one file per time interval

hello
    Just says a friendly Hello.Returns a literal string output with Hello plus the inputed name.

decompose_flow_vectors
    Process an indexed flow direction n

## Generate Climos

This process runs [generate_climos](https://github.com/pacificclimate/climate-explorer-data-prep/blob/master/README.md#generate_climos-generate-climatological-means), which creates files with climatological means/standard deviations of input data from a netcdf file. The `dry run` outputs metadata information about the input.

In [5]:
# Check info on `generate_climos` process
thunderbird.generate_climos?

Signature:
thunderbird.generate_climos(
    netcdf,
    operation,
    dry_run=None,
    convert_longitudes=True,
    split_vars=True,
    split_intervals=True,
    climo=None,
    resolutions=None,
)
Docstring:
Generate files containing climatological means from input files of daily, monthly, or yearly data that adhere to the PCIC metadata standard (and consequently to CMIP5 and CF standards).

Parameters
----------
netcdf : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    NetCDF file
operation : {'mean', 'std'}string
    Operation to perform on the datasets
climo : {'6190', '7100', '8110', '2020', '2050', '2080'}string
    Year ranges
resolutions : {'all', 'yearly', 'seasonal', 'monthly'}string
    Temporal Resolutions
convert_longitudes : boolean
    Transform longitude range from [0, 360) to [-180, 180)
split_vars : boolean
    Generate a separate file for each dependent variable in the file
split_intervals : boolean
    Generate a separate file f

In [6]:
# Dry run process
netcdf = 'https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/fdd_seasonal_CanESM2_rcp85_r1i1p1_1951-2100.nc'
operation = 'mean'
climo = '6190'
resolutions = 'yearly'
dry_run = True

dry_output = thunderbird.generate_climos(
    netcdf=netcdf, 
    operation=operation, 
    climo=climo, 
    resolutions=resolutions, 
    dry_run=dry_run
)

In [7]:
# Process dry run output
req = requests.get(dry_output.get()[0])
print(req.content.decode('utf-8'))

Dry Run
File: https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/fdd_seasonal_CanESM2_rcp85_r1i1p1_1951-2100.nc
climo_periods: {'6190'}
project: CMIP5
institution: PCIC
model: CanESM2
emissions: historical, rcp85
run: r1i1p1
dependent_varnames: ['fdd']
time_resolution: seasonal
is_multi_year_mean: False



In [8]:
# generate climos
output = thunderbird.generate_climos(
    netcdf=netcdf, 
    operation=operation, 
    climo=climo, 
    resolutions=resolutions, 
    dry_run=False
)

In [9]:
# Process normal output
req = requests.get(output.get()[0])
BeautifulSoup(BeautifulSoup(req.content.decode('utf-8')).prettify()).metaurl

<metaurl mediatype="application/x-netcdf">
     http://localhost:5001/outputs/f91e589e-b7cf-11ea-a36c-dc71961151bb/fdd_aClimMean_BCCAQ_CanESM2_historical+rcp85_r1i1p1_19610101-19901231_Canada.nc
    </metaurl>

## Split Merged Climos

This process runs [split_merged_climos](https://github.com/pacificclimate/climate-explorer-data-prep/blob/master/README.md#split_merged_climos-split-climo-means-files-into-per-interval-files-month-season-year), which splits climatological means files into one file per time interval.

In [10]:
# Check info on `split_merged_climos` process
thunderbird.split_merged_climos?

Signature: thunderbird.split_merged_climos(netcdf=None, loglevel='INFO')
Docstring:
Split climo means files into one file per time interval

Parameters
----------
netcdf : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    NetCDF files to process
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output : ComplexData:mimetype:`application/metalink+xml; version=4.0`
    Metalink object between output files
File:      ~/github/pcic/thunderbird/</home/nrados/github/pcic/thunderbird/notebooks/demo_venv/lib/python3.6/site-packages/birdy/client/base.py-3>
Type:      method


In [11]:
# Test local and opendap files
tasmax_climos_local = resource_filename('tests', 'data/tiny_downscaled_tasmax_climos.nc')
hydromodel_climos_opendap = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/tiny_hydromodel_gcm_climos.nc"

output = thunderbird.split_merged_climos([tasmax_climos_local, hydromodel_climos_opendap])

In [12]:
req = requests.get(output.get()[0])
BeautifulSoup(BeautifulSoup(req.content.decode('utf-8')).prettify()).find_all('metaurl')

[<metaurl mediatype="application/x-netcdf">
      http://localhost:5001/outputs/fc584a2e-b7cf-11ea-a36c-dc71961151bb/tasmax_mClim_BCCAQ2_ACCESS1-0_historical+rcp45_r1i1p1_19610101-19901231.nc
     </metaurl>,
 <metaurl mediatype="application/x-netcdf">
      http://localhost:5001/outputs/fc584a2f-b7cf-11ea-a36c-dc71961151bb/tasmax_sClim_BCCAQ2_ACCESS1-0_historical+rcp45_r1i1p1_19610101-19901231.nc
     </metaurl>,
 <metaurl mediatype="application/x-netcdf">
      http://localhost:5001/outputs/fc584a30-b7cf-11ea-a36c-dc71961151bb/tasmax_aClim_BCCAQ2_ACCESS1-0_historical+rcp45_r1i1p1_19610101-19901231.nc
     </metaurl>,
 <metaurl mediatype="application/x-netcdf">
      http://localhost:5001/outputs/fc584a31-b7cf-11ea-a36c-dc71961151bb/BASEFLOW+EVAP+GLAC_AREA_BAND+GLAC_MBAL_BAND+RUNOFF+SWE_BAND_mClim_VICGL+RGM+HydroCon_ACCESS1-0_historical+rcp45_r1i1p1_19840101-19951231.nc
     </metaurl>,
 <metaurl mediatype="application/x-netcdf">
      http://localhost:5001/outputs/fc584a32-b7cf-11ea-

## Update Metadata

This process runs [update_metadata](https://github.com/pacificclimate/climate-explorer-data-prep/blob/master/README.md#update_metadata-update-metadata-in-a-netcdf-file), which updates metadata from an input file according to instructions provided by a string or `.yaml` file. Rather than a metalink, the output is the updated netcdf file.

In [13]:
# Check info on `update_metadata` process
thunderbird.update_metadata?

Signature: thunderbird.update_metadata(netcdf, updates=None)
Docstring:
Update file containing missing, invalid, or incorrectly named global or variable metadata attributes

Parameters
----------
netcdf : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    NetCDF file
updates : string
    The filepath of an updates file that specifies what to do to the metadata it finds in the NetCDF file

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    Output Netcdf Files
File:      ~/github/pcic/thunderbird/</home/nrados/github/pcic/thunderbird/notebooks/demo_venv/lib/python3.6/site-packages/birdy/client/base.py-2>
Type:      method


In [14]:
updates = resource_filename("tests", "metadata-conversion/simple-conversion.yaml")
netcdf = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/gdd_annual_CanESM2_rcp85_r1i1p1_1951-2100.nc"
output = thunderbird.update_metadata(updates = updates, netcdf = netcdf)

In [15]:
print(output.get())

update_metadataResponse(
    output='http://localhost:5001/outputs/ffe82fa6-b7cf-11ea-a36c-dc71961151bb/gdd_annual_CanESM2_rcp85_r1i1p1_1951-2100_copy.nc'
)


## Generate Prsn

This process runs [generate_prsn](https://github.com/pacificclimate/climate-explorer-data-prep/blob/master/README.md#generate_prsn-generate-snowfall-file), which generates a `snowfall_flux` file from precipitation, tasmin, and tasmax input files.

In [16]:
# Check info on `generate_prsn` process
thunderbird.generate_prsn?

Signature:
thunderbird.generate_prsn(
    prec,
    tasmin=None,
    tasmax=None,
    dry_run=None,
    chunk_size=100,
    output_file='None',
    loglevel='INFO',
)
Docstring:
Generate precipitation as snow file from precipitation and minimum/maximum temperature data

Parameters
----------
prec : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Precipitation file to process
tasmin : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Tasmin file to process
tasmax : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Tasmax file to process
chunk_size : integer
    Number of time slices to be read/written at a time
output_file : string
    Optional custom name of output file
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level
dry_run : boolean
    Checks file to ensure compatible with process

Returns
-------
output : ComplexData:mimetype:`applic

In [17]:
# Test local and opendap files
pr_file_local = resource_filename("tests", "data/pr_week_test.nc")
tasmin_file_opendap = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/tasmin_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc"
tasmax_file_opendap = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/tasmax_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc"

dry_output = thunderbird.generate_prsn(pr_file_local, tasmin_file_opendap, tasmax_file_opendap, chunk_size=50, dry_run=True)

In [18]:
req = requests.get(dry_output.get()[0])
print(req.content.decode('utf-8'))

Dry Run
File: /tmp/pywps_process_5_3uas1l/pr_week_test.nc
project: CMIP5
model: NorESM1-M
institute: PCIC
experiment: historical,rcp26
ensemble_member: r1i1p1
dependent_varnames: ['pr']
File: https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/tasmin_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc
project: CMIP5
model: NorESM1-M
institute: PCIC
experiment: historical,rcp26
ensemble_member: r1i1p1
dependent_varnames: ['tasmin']
File: https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/tasmax_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc
project: CMIP5
model: NorESM1-M
institute: PCIC
experiment: historical,rcp26
ensemble_member: r1i1p1
dependent_varnames: ['tasmax']



In [ ]:
output = thunderbird.generate_prsn(pr_file_local, tasmin_file_opendap, tasmax_file_opendap, chunk_size=50, dry_run=False, output_file="prsn_test_mixed.nc")

In [20]:
req = requests.get(output.get()[0])
BeautifulSoup(BeautifulSoup(req.content.decode('utf-8')).prettify()).metaurl

<metaurl mediatype="application/x-netcdf">
     http://localhost:5001/outputs/bf4d1410-b712-11ea-afd0-c86000e3f2fd/prsn_test_mixed.nc
    </metaurl>

## Decompose Flow Vectors

This process runs [decompose_flow_vectors](https://github.com/pacificclimate/climate-explorer-data-prep#decompose_flow_vectors-create-normalized-unit-vector-fields-from-a-vic-routing-file), which writes to outfile a netCDF containing normalized vector arrays generated from variable in infile. It does not change infile or copy any other variables or axes to outfile.

In [ ]:
# Check info on `generate_prsn` process
thunderbird.decompose_flow_vectors?

In [ ]:
flow_vectors_file = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/sample_flow_parameters.nc"
variable = "Flow_Direction"
dest_file = "output.nc"
output = thunderbird.decompose_flow_vectors(netcdf=flow_vectors_file, variable=variable, dest_file=dest_file)

In [ ]:
output.get()